In [61]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_img_path = 'data/train_old'
val_img_path = 'data/test_old'
x_train = []
y_train = []
x_val = []
y_val = []

In [3]:
def load_path(img_path):
    _path = []
    lables = []
    with open(img_path, 'r') as file:
        # 将文本文件的数据读入到path_data
        path_data = file.readlines()
        for i in range(0, len(path_data)):
            # 将字符串格式转换为正常格式
            path_data[i] = path_data[i].split()
            _path.append(path_data[i][0])
            lables.append(int(path_data[i][1]))
        x = np.array(_path)
        y = np.array(lables)
        return x, y

In [4]:
x_train, y_train = load_path(train_img_path)
data_num = len(x_train)

In [5]:
def path_to_data(path, size=None):
    if size != None:
        return cv2.resize(cv2.imread(path), size)
    else:
        return cv2.resize(cv2.imread(path), (224, 224))

In [6]:
def generate_batch(x, y, batch_size=128):
    for start in range(0, data_num):
        x_batch = x[start: start + batch_size]
        y_batch = y[start: start + batch_size]
        start += batch_size

        _x_batch = []
        for i in range(x_batch.shape[0]):
            _x_batch += [path_to_data(x_batch[i]), (224, 224)]
        x_batch = np.array(_x_batch)
        yield x_batch, y_batch

In [11]:
print(generate_batch(x_train, y_train, 128))
print(x_train[0], y_train[0])

<generator object generate_batch at 0x7f588a27ad58>
data/track_data_new/no_daocha/jinan_20150921_no_daocha_left_00273.jpg 0


In [344]:
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, confusion_matrix
data = pd.read_csv('data.csv')
D = 'D'
N = 'N'
ROM = 'ROM'
L = 'L'
V = 'V'
rock = 'rock'

# 将带有中午属性的词转换为数值
listUniq = data.ix[:,'drill'].unique()
for j in range(len(listUniq)):
    data.ix[:,'drill'] = data.ix[:,'drill'].apply(lambda x:j if x==listUniq[j] else x)
listUniq = data.ix[:,'rock'].unique()
for j in range(len(listUniq)):
    data.ix[:,'rock'] = data.ix[:,'rock'].apply(lambda x:j if x==listUniq[j] else x)
    
# 进行one_hot编码
tempdata = data[['drill']]
enc = preprocessing.OneHotEncoder()
enc.fit(tempdata)

# 先将one-hot编码的结果是比较奇怪的，最好是先转换成二维数组
tempdata = enc.transform(tempdata).toarray()
print(tempdata)
print('取值范围整数个数：',enc.n_values_)

#再将二维数组转换为DataFrame，记得这里会变成多列
tempdata = pd.DataFrame(tempdata,columns=['house']*len(tempdata[0]))
print(tempdata)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]
取值范围整数个数： [5]
     house  house  house  house  house
0      1.0    0.0    0.0    0.0    0.0
1      0.0    1.0    0.0    0.0    0.0
2      1.0    0.0    0.0    0.0    0.0
3      1.0    0.0    0.0    0.0    0.0
4      1.0    0.0    0.0    0.0    0.0
5      0.0    1.0    0.0    0.0    0.0
6      0.0    1.0    0.0    0.0    0.0
7      0.0    1.0    0.0    0.0    0.0
8      1.0    0.0    0.0    0.0    0.0
9      0.0    1.0    0.0    0.0    0.0
10     1.0    0.0    0.0    0.0    0.0
11     0.0    1.0    0.0    0.0    0.0
12     1.0    0.0    0.0    0.0    0.0
13     0.0    1.0    0.0    0.0    0.0
14     1.0    0.0    0.0    0.0    0.0
15     0.0    1.0    0.0    0.0    0.0
16     1.0    0.0    0.0    0.0    0.0
17     1.0    0.0    0.0    0.0    0.0
18     0.0    1.0    0.0    0.0    0.0
19     1.0    0.0    0.0    0.0    0.0
20     0.0    1.0    0.0    0.0    0.0
21     0.0    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [361]:
x_columns = [x for x in data.columns if x in [D, N, ROM, L, V]]
X = np.array(pd.concat([tempdata, data[x_columns]],axis=1))
# X = np.array(data[x_columns])
X.resize([X.shape[1], 204])
y = np.array([data[rock]])
means = np.mean(X)
stedvs = np.std(X)
# X = (X - means) / stedvs
print(X)
# X = (X - X.min()) / (X.max() - X.min())
Y = np.array([data['rock']])
print(Y)

[[1.   1.   0.   ... 0.87 2.31 1.77]
 [0.   0.   0.   ... 3.28 1.06 2.5 ]
 [1.   1.   0.   ... 0.13 1.84 2.03]
 ...
 [0.   1.   0.   ... 2.83 3.19 3.06]
 [1.   0.   1.   ... 2.28 6.85 0.92]
 [0.   1.   1.   ... 2.87 2.17 3.32]]
[[0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 2 1 1 1 0 0 0 0 1 0 0 0 1 1 0 0
  1 3 3 1 1 3 3 3 1 3 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 2 1 1 1 1 1 0 0 0
  1 1 1 1 1 3 1 1 0 0 0 1 1 0 1 1 0 0 0 2 1 2 1 1 0 0 3 3 3 1 1 1 0 0 0 0
  0 2 1 0 2 0 1 0 0 0 3 0 2 0 0 3 2 1 2 0 1 3 1 0 1 3 2 1 3 3 3 3 3 0 0 0
  1 1 1 0 0 0 0 0 2 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 2 3 3 3 0 1 1
  0 0 0 0 0 0 0 0 0 0 1 1 1 2 1 1 1 0 0 0 2 1 3 1]]


In [172]:
# 构建神经网络结构
def layer_sizes(X, Y):
    """
    参数：
        X ：输入数据集，维度为（输入的数量，训练/测试的数量）
        Y ：标签，维度为（输出的数量，训练/测试数量）
    
    返回：
        n_x ：输入层数量
        n_y : 输出层数量
        n_h : 隐藏层数量
    """
    n_x = X.shape[0] # 输入层
    n_h = 4 # 隐藏层
    n_y = Y.shape[0] #输出层
    
    return (n_x, n_h, n_y)

In [173]:
# 初始化模型参数
def initialize_parameters(n_x, n_h, n_y):
    """
    参数：
        n_x 输入节点的数量
        n_h 隐藏节点的数量
        n_y 输出节点的数量
    返回：
        parameters 包含参数的字典
            W1 权重矩阵，维度为（n_h, n_x）
            b1 偏向量，维度为（n_h, 1）
            W2 权重矩阵，维度为（n_y, n_h）
            b1 偏向量，维度为（n_y, 1）
    """
    np.random.seed(2) # 指定一个随机种子
    W1 = np.random.randn(n_h, n_x) * 0.01 # 随机初始化一个维度为（n_h, n_x）的矩阵
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros(shape=(n_y, 1))
    
    # 使用断言判断格式是否正确
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1" : W1,
                  "b1" : b1,
                  "W2" : W2,
                  "b2" : b2}
    return parameters

In [174]:
def forward_propagation(X, parameters):
    """
    参数：
        X 维度为（n_x, m）的输入数据
        parameters 初始化函数（initialize_parameters）的输出
        
    返回：
        A2 使用sigmide()函数计算的第二次激活后的数值
        cache 包含“Z1”，“Z2”，“A1”和“A2”的字典类型变量
    """
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
    # 前向传播计算A2
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    # 使用断言判断格式
    assert(A2.shape == (1, X.shape[1]))
    cache = {"Z1" : Z1,
             "Z2" : Z2,
             "A1" : A1,
             "A2" : A2}
    return (A2, cache)

In [280]:
# 计算交叉熵损失
def compute_cost(A2, Y, parameters):
    """
    参数：
        A2 使用sigmoid()函数计算的第二次激活后的值
        Y 标签向量，维度为（1， 数量）
        parameters 包含W1,b1,W2和b2的字典类型的变量
    返回：
        成本 交叉熵损失
    """
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
    # 计算成本
    logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1-A2), (1-Y))
    cost = - np.sum(logprobs) / m
    cost = float(np.squeeze(cost))
    
    assert(isinstance(cost, float))
    return cost

In [274]:
# 反向传播
def backward_propagation(parameters, cache, X, Y):
    """
    参数：
        parameters 包含参数的一个字典型变量
        cache 包含“Z1”，“A1”，“Z2”和“A2”的字典类型的变量
        X 输入数据，维度为（2， 数量）
        Y 标签，维度为（1，数量）
    返回
        grads 包含W和b的一个字典型变量
    """
    m = X.shape[1]
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    A1 = cache["A1"]
    A2 = cache["A2"]
    
    dZ2 = A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    grads = {"dW1" : dW1,
             "db1" : db1,
             "dW2" : dW2,
             "db2" : db2}
    
    return grads

In [275]:
# 更新参数
def update_parameters(parameters, grads, learning_rate = 0.01):
    """
    参数：
        parameters 包含参数的字典类型变量
        grads 包含导数值的字典类型变量
        learning_rate 学习率
    返回：
        parameters 包含更新参数后的字典类型变量
    """
    
    W1, W2 = parameters["W1"], parameters["W2"]
    b1, b2 = parameters["b1"], parameters["b2"]
    
    dW1, dW2 = grads["dW1"], grads["dW2"]
    db1, db2 = grads["db1"], grads["db2"]
    
    W1 = W1 - learning_rate * dW1
    W2 = W2 - learning_rate * dW2
    b1 = b1 - learning_rate * db1
    b2 = b2 - learning_rate * db2
    
    parameters={"W1" : W1,
                "b1" : b1,
                "W2" : W2,
                "b2" : b2}
    return parameters

In [276]:
def nn_model(X, Y, n_h, num_iterations, print_cost = False):
    """
    参数：
        X 数据集，维度为（2， 示例数）
        Y 标签，维度为（1， 示例数）
        n_h 隐藏层的数量
        num_iterations 梯度下降循环中的循环次数
        print_cost 如果为True，则每1000次迭代打印一次成本数值
    返回：
        parameters 学习模型的参数，用它们来进行预测
    """
    np.random.seed(3) # 指定随机种子
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    for i in range(num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y, parameters)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grads, learning_rate=0.005)
        
        if print_cost:
            if i%1000 == 0:
                print("第 ", i, "次循环，lost为" + str(cost))
    return parameters

In [277]:
# 预测
def predict(parameters, X):
    """
    参数：
        parameters 包含参数的字典类型的变量
        X 输入数据（n_x, m）
    返回：
        prediction 模型预测的向量（红色：0/蓝色：1）
    """
    A2, cache = forward_propagation(X, parameters)
    prediction = np.round(A2)
    
    return prediction

In [362]:
# 正式运行
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s
parameters = nn_model(X, Y, n_h=4, num_iterations=50000,print_cost=True)
predictions = predict(parameters, X)
accuracy = precision_score(Y[0], predictions[0], average='micro')*100
print("准确率：{} %" .format(accuracy))

第  0 次循环，lost为0.6842114074142202
第  1000 次循环，lost为0.3264313395846262
第  2000 次循环，lost为0.32831877139820115
第  3000 次循环，lost为0.28471908477146934
第  4000 次循环，lost为0.27301584460614314
第  5000 次循环，lost为0.21002688236507153
第  6000 次循环，lost为0.1598817366481211
第  7000 次循环，lost为0.10728698473875664
第  8000 次循环，lost为0.05174990823928235
第  9000 次循环，lost为-0.00872881451005084
第  10000 次循环，lost为-0.07573264323875746
第  11000 次循环，lost为-0.07940193257707871
第  12000 次循环，lost为-0.17309869805181946
第  13000 次循环，lost为-0.2902741790741063
第  14000 次循环，lost为-0.3878183641409032
第  15000 次循环，lost为-0.4870426138718867
第  16000 次循环，lost为-0.5880853573225145
第  17000 次循环，lost为-0.6916338254807224
第  18000 次循环，lost为-0.7982343976688403
第  19000 次循环，lost为-0.9083090302106729
第  20000 次循环，lost为-1.0220323004180194
第  21000 次循环，lost为-1.1392040955800904
第  22000 次循环，lost为-1.2590234368001054
第  23000 次循环，lost为-1.3807892556555084


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


第  24000 次循环，lost为-inf
第  25000 次循环，lost为-inf
第  26000 次循环，lost为-inf


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()


第  27000 次循环，lost为nan
第  28000 次循环，lost为nan
第  29000 次循环，lost为nan
第  30000 次循环，lost为nan
第  31000 次循环，lost为nan
第  32000 次循环，lost为nan
第  33000 次循环，lost为nan
第  34000 次循环，lost为nan
第  35000 次循环，lost为nan
第  36000 次循环，lost为nan


/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


第  37000 次循环，lost为nan
第  38000 次循环，lost为nan
第  39000 次循环，lost为nan
第  40000 次循环，lost为nan
第  41000 次循环，lost为nan
第  42000 次循环，lost为nan
第  43000 次循环，lost为nan
第  44000 次循环，lost为nan
第  45000 次循环，lost为nan
第  46000 次循环，lost为nan
第  47000 次循环，lost为nan
第  48000 次循环，lost为nan
第  49000 次循环，lost为nan
准确率：48.529411764705884 %


In [365]:
hidden_layer_sizes = [1, 2, 3, 4, 5, 20] #隐藏层数量
for i, n_h in enumerate(hidden_layer_sizes):
    parameters = nn_model(X, Y, n_h, num_iterations=20000)
    predictions = predict(parameters, X)
    matrix = confusion_matrix(Y[0], predictions[0])
    print(matrix)
#     accuracy = float((np.dot(Y, predictions.T) + np.dot(1 - Y, 1 - predictions.T)) / float(Y.size) * 100)
#     accuracy = np.sum(np.not_equal(Y, predictions))/float(Y.size)
    accuracy = precision_score(Y[0], predictions[0], average='micro')*100
    print ("隐藏层的节点数量： {}， 准确率: {} %".format(n_h, accuracy))

[[ 0 85  0  0]
 [ 0 82  0  0]
 [ 0 14  0  0]
 [ 0 23  0  0]]
隐藏层的节点数量： 1， 准确率: 40.19607843137255 %
[[ 4 81  0  0]
 [ 1 81  0  0]
 [ 0 14  0  0]
 [ 0 23  0  0]]
隐藏层的节点数量： 2， 准确率: 41.66666666666667 %
[[17 68  0  0]
 [ 5 77  0  0]
 [ 0 14  0  0]
 [ 1 22  0  0]]
隐藏层的节点数量： 3， 准确率: 46.07843137254902 %
[[18 67  0  0]
 [ 3 79  0  0]
 [ 0 14  0  0]
 [ 2 21  0  0]]
隐藏层的节点数量： 4， 准确率: 47.549019607843135 %
[[20 65  0  0]
 [ 6 76  0  0]
 [ 0 14  0  0]
 [ 2 21  0  0]]
隐藏层的节点数量： 5， 准确率: 47.05882352941176 %


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()


[[17 68  0  0]
 [ 3 79  0  0]
 [ 0 14  0  0]
 [ 1 22  0  0]]
隐藏层的节点数量： 20， 准确率: 47.05882352941176 %
